# __THIS NOTEBOOK IS TO BE USED FOR IBM DATA SCIENCE CAPSTONE PROJECT__


In [1]:
#Import all necessary libraries/packages
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
import folium # map rendering library
import urllib.request
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests # library to handle requests
print('Libraries imported.')

Libraries imported.


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## 1. Retrieve the Foursquare data and put it into a pandas dataframe:

In [33]:
CLIENT_ID = 'K2OBCTVKCRTDK2SSULLRULJ0ZH2GPGEN35FX5MSYF023SPPV' # your Foursquare ID
CLIENT_SECRET = 'Z2EG1JCMBCPBXV324EPLA1HM3NYQI30GAAPN3MQVJGQETLIM' # your Foursquare Secret
VERSION = '20180604'
radius = 10000
LIMIT = 100

In [34]:
address = '400 Welton St, Denver, CO'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

39.7546069 -104.9783125


In [35]:
search_query = 'restaurant'

In [36]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=K2OBCTVKCRTDK2SSULLRULJ0ZH2GPGEN35FX5MSYF023SPPV&client_secret=Z2EG1JCMBCPBXV324EPLA1HM3NYQI30GAAPN3MQVJGQETLIM&ll=39.7546069,-104.9783125&v=20180604&query=restaurant&radius=10000&limit=100'

In [104]:
results = requests.get(url).json()

In [38]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

C:\Users\Jake - Alienware\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,...,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,location.crossStreet,location.neighborhood,venuePage.id
0,4b75a7bff964a5208c1a2ee3,La Fiesta Mexican Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1592846842,False,2340 Champa St,39.753410,-104.984636,"[{'label': 'display', 'lat': 39.75341022800047...",557,...,"[2340 Champa St, Denver, CO 80205, United States]",555642,https://www.grubhub.com/restaurant/la-fiesta-2...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN,NaN,NaN
1,3fd66200f964a520b4ec1ee3,Rio Grande Mexican Restaurant,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",v-1592846842,False,1525 Blake St,39.750223,-105.000036,"[{'label': 'display', 'lat': 39.75022256969623...",1922,...,"[1525 Blake St (btwn 17th St & 15th St), Denve...",306685,https://www.grubhub.com/restaurant/rio-grande-...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,btwn 17th St & 15th St,LoDo,NaN
2,4c06fc11b4aa0f475e056562,Randolph's Restaurant & Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1592846842,False,1776 Grant St,39.744531,-104.983480,"[{'label': 'display', 'lat': 39.74453056889666...",1205,...,"[1776 Grant St (at 17th), Denver, CO 80203, Un...",NaN,NaN,NaN,NaN,NaN,NaN,at 17th,NaN,36169341
3,4a77ac3ff964a520fbe41fe3,The Corner Office Restaurant & Martini Bar,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1592846842,False,1401 Curtis St,39.745555,-104.997231,"[{'label': 'display', 'lat': 39.74555533418328...",1907,...,"[1401 Curtis St (at 14th St), Denver, CO 80202...",NaN,NaN,NaN,NaN,NaN,NaN,at 14th St,NaN,35396549
4,4cc60fb18062468804e23f2f,EDGE Restaurant And Bar At Four Seasons Hotel ...,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",v-1592846842,False,1111 14th St,39.746367,-104.997919,"[{'label': 'display', 'lat': 39.74636682978978...",1912,...,"[1111 14th St (Arapahoe), Denver, CO 80202, Un...",NaN,NaN,NaN,NaN,NaN,NaN,Arapahoe,NaN,NaN


## 1a. Filter only the relevant columns:

In [105]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,La Fiesta Mexican Restaurant,Mexican Restaurant,2340 Champa St,39.753410,-104.984636,"[{'label': 'display', 'lat': 39.75341022800047...",557,80205,US,Denver,CO,United States,"[2340 Champa St, Denver, CO 80205, United States]",NaN,NaN,4b75a7bff964a5208c1a2ee3
1,Rio Grande Mexican Restaurant,Mexican Restaurant,1525 Blake St,39.750223,-105.000036,"[{'label': 'display', 'lat': 39.75022256969623...",1922,80202,US,Denver,CO,United States,"[1525 Blake St (btwn 17th St & 15th St), Denve...",btwn 17th St & 15th St,LoDo,3fd66200f964a520b4ec1ee3
2,Randolph's Restaurant & Bar,American Restaurant,1776 Grant St,39.744531,-104.983480,"[{'label': 'display', 'lat': 39.74453056889666...",1205,80203,US,Denver,CO,United States,"[1776 Grant St (at 17th), Denver, CO 80203, Un...",at 17th,NaN,4c06fc11b4aa0f475e056562
3,The Corner Office Restaurant & Martini Bar,American Restaurant,1401 Curtis St,39.745555,-104.997231,"[{'label': 'display', 'lat': 39.74555533418328...",1907,80202,US,Denver,CO,United States,"[1401 Curtis St (at 14th St), Denver, CO 80202...",at 14th St,NaN,4a77ac3ff964a520fbe41fe3
4,EDGE Restaurant And Bar At Four Seasons Hotel ...,Steakhouse,1111 14th St,39.746367,-104.997919,"[{'label': 'display', 'lat': 39.74636682978978...",1912,80202,US,Denver,CO,United States,"[1111 14th St (Arapahoe), Denver, CO 80202, Un...",Arapahoe,NaN,4cc60fb18062468804e23f2f


## 1b. Get ratings for all venues in dataframe: 

In [90]:
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4c0ba8406071a5933484e132?client_id=K2OBCTVKCRTDK2SSULLRULJ0ZH2GPGEN35FX5MSYF023SPPV&client_secret=Z2EG1JCMBCPBXV324EPLA1HM3NYQI30GAAPN3MQVJGQETLIM&v=20180604'

In [108]:
result = requests.get(url).json()
#print(result['response']['venue'].keys())
#result['response']['venue']

## 1c. Find the associated ratings for each venue category:

In [92]:
for i, j in zip(dataframe_filtered['id'], dataframe_filtered['categories']):
    venue_id = i
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'], ":  ", j )
    except:
        print('This venue has not been rated yet.', ":  ", j )



7.6 :   Mexican Restaurant
7.6 :   Mexican Restaurant
6.8 :   American Restaurant
7.6 :   American Restaurant
7.4 :   Steakhouse
This venue has not been rated yet. :   Food Service
6.1 :   Restaurant
6.7 :   Indian Restaurant
5.9 :   Lounge
8.6 :   Japanese Restaurant
7.3 :   Seafood Restaurant
This venue has not been rated yet. :   Mexican Restaurant
6.3 :   Kitchen Supply Store
7.1 :   Pizza Place
This venue has not been rated yet. :   Food
This venue has not been rated yet. :   African Restaurant
6.5 :   Mexican Restaurant
7.5 :   Mexican Restaurant
7.4 :   Brewery
This venue has not been rated yet. :   Restaurant
This venue has not been rated yet. :   Chinese Restaurant
7.1 :   Mexican Restaurant
6.6 :   Ethiopian Restaurant
This venue has not been rated yet. :   Gastropub
This venue has not been rated yet. :   Mexican Restaurant
8.7 :   Mexican Restaurant
7.5 :   Ethiopian Restaurant
This venue has not been rated yet. :   Waste Facility
This venue has not been rated yet. :   Mexic

In [77]:
mr_avg = (7.6 + 7.6 + 6.5 + 7.5 + 7.1 + 8.7 + 6.7)/8
ar_avg = (6.8+7.6+4.7)/3
s_avg = 7.4
r_avg = 6.1
ir_avg = (6.7+7.5)/3
l_avg = 5.9
jr_avg = 8.6
sr_avg = 7.3
pp_avg = 7.1
b_avg = 7.4
er_avg = (6.6+7.5)/2
bs_avg = 8.0

## 1d. Filter out only restaurants that have at least 1 rating, and load into dataframe: 

In [99]:
data = {'Venue Category':  ['Mexican Restaurant', 'American Restaurant','Steakhouse','Restaurant',
                              'Indian Restaurant', 'Lounge', 'Japanese Restaurant', 'Seafood Restaurant',
                              'Pizza Place','Brewery',
                              'Ethiopian Restaurant', 'Breakfast Spot'],
        'Avg_Rating': [mr_avg, ar_avg, s_avg, r_avg, ir_avg, l_avg, jr_avg, sr_avg, pp_avg, b_avg, er_avg, bs_avg ]}

ratings_df = pd.DataFrame (data, columns = ['Venue Category','Avg_Rating'])

## 1e. Sort the venue categories by highest rating:

In [100]:
ratings_df.sort_values(by=['Avg_Rating'], ascending = False)

,Venue Category,Avg_Rating
6,Japanese Restaurant,8.600000
11,Breakfast Spot,8.000000
2,Steakhouse,7.400000
9,Brewery,7.400000
7,Seafood Restaurant,7.300000
8,Pizza Place,7.100000
10,Ethiopian Restaurant,7.050000
0,Mexican Restaurant,6.462500
1,American Restaurant,6.366667
3,Restaurant,6.100000


## 2. Map of all local restaurants in the dowtown Denver area:

In [101]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## 3. Explore trending venues in the area (areas with the highest foot traffic):

In [11]:

url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()

In [12]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [13]:
trending_venues_df

'No trending venues are available at the moment!'

In [14]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
        
        
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

## 4. Find the most common type of venue in downtown Denver:

In [102]:
def getNearbyVenues(names, latitudes, longitudes, radius=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [103]:
denver_venues = getNearbyVenues(names=dataframe_filtered['name'],
                                   latitudes=dataframe_filtered['lat'],
                                   longitudes=dataframe_filtered['lng']
                                  )

La Fiesta Mexican Restaurant
Rio Grande Mexican Restaurant
Randolph's Restaurant & Bar
The Corner Office Restaurant & Martini Bar
EDGE Restaurant And Bar At Four Seasons Hotel Denver
The Restaurant Source, a division of Bargreen Ellingson
Altitude Restaurant
Little India Restaurant - Champa Downtown
Pete's Satire Restaurant & Lounge
Domo Japanese Country Foods Restaurant
Denver Downtown Aquarium Restaurant
Mexico City Restaurant
Restaurant Depot
Lechuga's Italian Restaurant
Restaurant Connection Denver
D-Line Restaurant
Bella Vista Mexican Restaurant
Mexico City Restaurant
Liberati Restaurant & Brewery
Solutions Lounge and Restaurant
New Peking Chinese Restaurant
Benny's Restaurant and Tequila Bar
Ethiopian Restaurant
Restaurant Burke
Las Delicias Authentic Mexican Restaurant
Santiago's Mexican Restaurant
Abyssinia Ethiopian Restaurant
Restaurant Rescue, Inc., of Colorado
El Charrito Mexican Restaurant
Foraged Restaurant
1876 Restaurant
Camargos Tamales Restaurant
Far East Restaurant
M

In [120]:
denver_venues

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,La Fiesta Mexican Restaurant,39.753410,-104.984636,Mexican Restaurant
1,Little Owl Coffee,39.750452,-104.999656,Coffee Shop
2,ChoLon Bistro,39.750562,-104.999673,Asian Restaurant
3,Freshcraft,39.750012,-104.999685,American Restaurant
4,D'Corazon,39.750116,-104.999667,Mexican Restaurant
...,...,...,...,...
107,Body Massage Wellness Spa,39.744698,-104.990583,Spa
108,Grand Hyatt Denver,39.745936,-104.989333,Hotel
109,ink! Coffee,39.746151,-104.989807,Coffee Shop
110,Mici Handcrafted Italian,39.745580,-104.993900,Italian Restaurant


In [17]:
denver_venues['Venue Category'].value_counts()

Mexican Restaurant               11
American Restaurant              10
Bar                               6
Coffee Shop                       5
Sandwich Place                    4
Hotel                             4
Marijuana Dispensary              4
Thai Restaurant                   3
Sushi Restaurant                  2
Burger Joint                      2
Japanese Restaurant               2
Noodle House                      2
Brewery                           2
Seafood Restaurant                2
Italian Restaurant                2
Chinese Restaurant                2
Convenience Store                 2
Ethiopian Restaurant              2
Discount Store                    2
Spa                               2
Dive Bar                          2
Pizza Place                       2
Lounge                            2
Martial Arts Dojo                 1
Optical Shop                      1
Boutique                          1
Kitchen Supply Store              1
Grocery Store               

## 5. Clustering the venues in dowtown Denver: 

In [18]:
denver_onehot = pd.get_dummies(denver_venues[['Venue Category']], prefix="", prefix_sep="")
denver_onehot['Venue'] = denver_venues['Venue'] 

In [19]:
# get a list of columns
cols = list(denver_onehot)
# move the column to head of list using index, pop and insert
cols.insert(0, cols.pop(cols.index('Venue')))
cols

['Venue',
 'American Restaurant',
 'Aquarium',
 'Arcade',
 'Art Gallery',
 'Asian Restaurant',
 'BBQ Joint',
 'Bakery',
 'Bar',
 'Boutique',
 'Breakfast Spot',
 'Brewery',
 'Burger Joint',
 'Butcher',
 'Chinese Restaurant',
 'Cocktail Bar',
 'Coffee Shop',
 'Convenience Store',
 'Cosmetics Shop',
 'Dessert Shop',
 'Diner',
 'Discount Store',
 'Dive Bar',
 'Donut Shop',
 'Ethiopian Restaurant',
 'Food Service',
 'Fried Chicken Joint',
 'Furniture / Home Store',
 'Grocery Store',
 'Hotel',
 'Hotel Pool',
 'Indian Restaurant',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Kitchen Supply Store',
 'Lounge',
 'Marijuana Dispensary',
 'Martial Arts Dojo',
 'Mexican Restaurant',
 'New American Restaurant',
 'Noodle House',
 'Opera House',
 'Optical Shop',
 'Pizza Place',
 'Plaza',
 'Ramen Restaurant',
 'Recreation Center',
 'Sandwich Place',
 'Seafood Restaurant',
 'Shipping Store',
 'Spa',
 'Speakeasy',
 'Steakhouse',
 'Sushi Restaurant',
 'Thai Restaurant',
 'Theater',
 'Vegetarian / Vega

## 5a. Onehot encoding for cluster use:

In [20]:
denver_onehot = denver_onehot.reindex(columns= cols)

In [21]:
denver_grouped = denver_onehot.groupby('Venue').mean().reset_index()
denver_grouped

,Venue,American Restaurant,Aquarium,Arcade,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bar,Boutique,...,Sandwich Place,Seafood Restaurant,Shipping Store,Spa,Speakeasy,Steakhouse,Sushi Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant
0,7-Eleven,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9Thai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Abarrotes La Herradura,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abyssinia Ethiopian Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aloy Modern Thai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Yard House,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,denver recreational,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,il posto,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,ink! Coffee,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
num_top_venues = 5

for hood in denver_grouped['Venue']:
    temp = denver_grouped[denver_grouped['Venue'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 1

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Venue']
for ind in np.arange(num_top_venues):
    try:
        columns.append('Venue Type'.format(ind+1, indicators[ind]))
    except:
        columns.append('Venue Type'.format(ind+1))

# create a new dataframe
denver_venues_sorted = pd.DataFrame(columns=columns)
denver_venues_sorted['Venue'] = denver_grouped['Venue']

for ind in np.arange(denver_grouped.shape[0]):
    denver_venues_sorted.iloc[ind, 1:] = return_most_common_venues(denver_grouped.iloc[ind, :], num_top_venues)

denver_venues_sorted.head()

,Venue,Venue Type
0,7-Eleven,Convenience Store
1,9Thai,Thai Restaurant
2,Abarrotes La Herradura,Convenience Store
3,Abyssinia Ethiopian Restaurant,Ethiopian Restaurant
4,Aloy Modern Thai,Thai Restaurant


In [25]:
# set number of clusters
kclusters = 7

denver_grouped_clustering = denver_grouped.drop('Venue', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(denver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] .astype(int) 

array([0, 5, 0, 0, 5, 0, 3, 0, 2, 2])

In [29]:
del denver_venues_sorted['Cluster Labels']

In [30]:
# add clustering labels
denver_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

denver_merged = denver_venues

denver_merged = denver_merged.join(denver_venues_sorted.set_index('Venue'), on='Venue', how = 'right')

denver_merged.head() # check the last columns

,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,Venue Type
0,La Fiesta Mexican Restaurant,39.753410,-104.984636,Mexican Restaurant,2,Mexican Restaurant
1,Little Owl Coffee,39.750452,-104.999656,Coffee Shop,4,Coffee Shop
2,ChoLon Bistro,39.750562,-104.999673,Asian Restaurant,0,Asian Restaurant
3,Freshcraft,39.750012,-104.999685,American Restaurant,3,American Restaurant
4,D'Corazon,39.750116,-104.999667,Mexican Restaurant,2,Mexican Restaurant


In [176]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
integer = 1
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(denver_merged['Venue Latitude'], denver_merged['Venue Longitude'], denver_merged['Venue'], denver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 0:

In [197]:
cluster_0 = denver_merged.loc[denver_merged['Cluster Labels'] == 0, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]
cluster_0

,Venue,Venue Type
2,ChoLon Bistro,Asian Restaurant
6,Warwick Denver Hotel,Hotel
8,Ellie Caulkins Opera House,Opera House
9,DoubleTree by Hilton,Hotel
12,Four Seasons Spa,Spa
...,...,...
106,Amore Gelato & Crepes,Dessert Shop
107,Body Massage Wellness Spa,Spa
108,Grand Hyatt Denver,Hotel
110,Mici Handcrafted Italian,Italian Restaurant


In [ ]:
cluster_0['Venue Type'].value_counts()

## Cluster 1:

In [189]:
denver_merged.loc[denver_merged['Cluster Labels'] == 1, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
3,Freshcraft,American Restaurant
7,Randolph's Restaurant & Bar,American Restaurant
10,The Corner Office Restaurant & Martini Bar,American Restaurant
13,Nickel,American Restaurant
43,The Cheesecake Factory,American Restaurant
61,D Bar Denver,American Restaurant
76,Vesta Dipping Grill,American Restaurant
86,"ViewHouse Eatery, Bar & Rooftop",American Restaurant
91,Yard House,American Restaurant
100,West 29th Restaurant & Bar,American Restaurant


## Cluster 2:

In [190]:
denver_merged.loc[denver_merged['Cluster Labels'] == 2, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
22,Cheba Hut Toasted Subs,Sandwich Place
73,Subway,Sandwich Place
94,Potbelly Sandwich Shop,Sandwich Place


## Cluster 3:

In [191]:
denver_merged.loc[denver_merged['Cluster Labels'] == 3, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
1,Little Owl Coffee,Coffee Shop
11,Starbucks,Coffee Shop
96,Starbucks,Coffee Shop
62,Fluid Coffee Bar,Coffee Shop
109,ink! Coffee,Coffee Shop


## Cluster 4:

In [192]:
denver_merged.loc[denver_merged['Cluster Labels'] == 4, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
0,La Fiesta Mexican Restaurant,Mexican Restaurant
4,D'Corazon,Mexican Restaurant
5,Rio Grande Mexican Restaurant,Mexican Restaurant
36,Mexico City Restaurant,Mexican Restaurant
69,Mexico City Restaurant,Mexican Restaurant
41,Benny's Restaurant and Tequila Bar,Mexican Restaurant
63,Las Delicias,Mexican Restaurant
90,Bella Vista Mexican Restaurant,Mexican Restaurant
99,Camargos Tamales Restaurant,Mexican Restaurant


## Cluster 5:

In [193]:
denver_merged.loc[denver_merged['Cluster Labels'] == 5, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
33,Aloy Modern Thai,Thai Restaurant
74,9Thai,Thai Restaurant
75,Thai Monkey Club On Colfax,Thai Restaurant


## Cluster 6:

In [194]:
denver_merged.loc[denver_merged['Cluster Labels'] == 6, denver_merged.columns[[0] + list(range(5, denver_merged.shape[1]))]]

,Venue,Venue Type
37,Ignite Kitchen + Cocktails,Bar
70,Ignite Kitchen + Cocktails,Bar
51,First Draft Taproom & Kitchen,Bar
53,The Matchbox,Bar
55,Lustre Pearl,Bar
66,Mythology Distillery,Bar
